In [1]:
### Load packages
using LinearAlgebra
using Plots
using DataFrames, CSV, HypothesisTests
using Statistics
using Dates
using RDatasets, MultivariateStats # for PCA

In [8]:
mutable struct Kalman
    A
    G
    Q
    R
    E # New matrix for external variable
    k
    n
    cur_x_hat
    cur_sigma
end


# Initializes current mean and cov to zeros
function Kalman(A, G, Q, R, E )
    k = size(G, 1)
    n = size(G, 2)
    xhat = n == 1 ? zero(eltype(A)) : zeros(n)
    Sigma = n == 1 ? zero(eltype(A)) : zeros(n, n)
    return Kalman(A, G, Q, R, E, k, n, xhat, Sigma)
end

function set_state!(k::Kalman, x_hat, Sigma)
    k.cur_x_hat = x_hat
    k.cur_sigma = Sigma
    Nothing
end

### Our function doesn't work when we have some missing external regressors. 
function prior_to_filtered!(k::Kalman, y::AbstractVector, external = 1)
    # simplify notation
    G, R, E = k.G, k.R, k.E
    x_hat, Sigma = k.cur_x_hat, k.cur_sigma
    
    ### Change NaN into missing
    y = replace(y, NaN => missing)
    
    ### Deal with missing observation (missing)
    if size(y) != size(collect(skipmissing(y)))
        j = Array{Bool}(undef,length(y))
        l = ismissing.(y)
        for i in 1:length(y)
            j[i] = !l[i]
        end
        
        ## Change dimension of valid input, and matrices
        y = y[j]
        G = G[j,:]
        R = R[j,j]
        E = E[j,:]
    end
    
    # and then update
    A = Sigma*G'
    B = G*Sigma*G' + R
    M = A / B
    
    k.cur_x_hat = x_hat + M * (y - (G * x_hat) - (E * (external))) ### Add external variables
    k.cur_sigma = Sigma - M * G * Sigma
    Nothing
end

function filtered_to_forecast!(k::Kalman)
    # simplify notation
    A, Q = k.A, k.Q
    x_hat, Sigma = k.cur_x_hat, k.cur_sigma

    # and then update
    k.cur_x_hat = A * x_hat
    k.cur_sigma = A * Sigma * A' + Q
    Nothing
end

function log_likelihood(k::Kalman, y::AbstractVector, external = 1)
    # Simplify notation
    G, R, E = k.G, k.R, k.E
    x_hat, Sigma = k.cur_x_hat, k.cur_sigma
    
    ### Change NaN into missing
    y = replace(y, NaN => missing)
    
    ### Deal with missing observation (missing)
    if size(y) != size(collect(skipmissing(y)))
        j = Array{Bool}(undef,length(y))
        k = ismissing.(y)
        for i in 1:length(y)
            j[i] = !k[i]
        end
        ## Change dimension of valid input, and matrices
        y = y[j]
        G = G[j,:]
        R = R[j,j]
        E = E[j,:]
        println(y)
        println(G)
        println(R)
        println(E)
    end
    
    eta = y - G*x_hat - E*external # forecast error
    P = G*Sigma*G' + R # covariance matrix of forecast error
    logL = - (length(y)*log(2pi) + logdet(P) .+ eta'/P*eta)[1]/2
    return logL
end

function compute_loglikelihood(kn::Kalman, y::AbstractMatrix, external = 1)
    T = size(y, 2)
    logL = 0
    
    
    # forecast and update
    if external == 1
        for t in 1:T
            logL = logL + log_likelihood(kn, y[:,t])
            update!(kn, y[:, t])
        end
    else
        for t in 1:T
            logL = logL + log_likelihood(kn, y[:,t], external[:,t])
            update!(kn, y[:, t], external[:,t])
        end
    end

    return logL
end

function update!(k::Kalman, y, external = 1)
    prior_to_filtered!(k, y, external)
    filtered_to_forecast!(k)
    Nothing
end

function smooth(kn::Kalman, y::DataFrame, external::DataFrame)
    G, R = kn.G, kn.R

    T = size(y, 2)
    n = kn.n
    x_filtered = Matrix{Float64}(undef, n, T)
    sigma_filtered = Array{Float64}(undef, n, n, T)
    sigma_forecast = Array{Float64}(undef, n, n, T)
    logL = 0
    # forecast and update
    for t in 1:T
        logL = logL + log_likelihood(kn, y[:, t], external[:,t])
        prior_to_filtered!(kn, y[:, t], external[:,t])
        x_filtered[:, t], sigma_filtered[:, :, t] = kn.cur_x_hat, kn.cur_sigma
        filtered_to_forecast!(kn)
        sigma_forecast[:, :, t] = kn.cur_sigma
    end
    # smoothing
    x_smoothed = copy(x_filtered)
    sigma_smoothed = copy(sigma_filtered)
    for t in (T-1):-1:1
        x_smoothed[:, t], sigma_smoothed[:, :, t] =
            go_backward(kn, x_filtered[:, t], sigma_filtered[:, :, t],
                        sigma_forecast[:, :, t], x_smoothed[:, t+1],
                        sigma_smoothed[:, :, t+1])
    end

    return x_smoothed, logL, sigma_smoothed
end

function go_backward(k::Kalman, x_fi::Vector,
                     sigma_fi::Matrix, sigma_fo::Matrix,
                     x_s1::Vector, sigma_s1::Matrix)
    A = k.A
    temp = sigma_fi*A'/sigma_fo
    x_s = x_fi + temp*(x_s1-A*x_fi)
    sigma_s = sigma_fi + temp*(sigma_s1-sigma_fo)*temp'
    return x_s, sigma_s
end

go_backward (generic function with 1 method)

In [9]:
function check_kalman(k::Kalman, y::DataFrame)
    ### Initiate error counter
    counter = 0
    
    ### Checking type of input
    if typeof(G) != Array{Float64,1} && typeof(G) != Array{Float64,2} &&
        typeof(G) != Array{Int64,1} && typeof(G) != Array{Int64,2}
        return "G should be a matrix."
    end
    n = size(G,1)
    r = size(G,2)
    
    if r == 1
        if typeof(A) != Array{Float64,1} && typeof(A) != Array{Int64,1}
            println("A has to be 1x1 matrix.")
            counter = counter + 1
        end
        if typeof(Q) != Array{Float64,1} && typeof(Q) != Array{Int64,1}
            println("Q has to be 1x1 matrix.")
            counter = counter + 1
        end
    else
        if typeof(A) != Array{Float64,2} && typeof(A) != Array{Int64, 2}
            println("A has to be a matrix.")
            counter = counter + 1 
        end
        if typeof(Q) != Array{Float64,2} && typeof(Q) != Array{Int64, 2}
            println("Q has to be a matrix.")
            counter = counter + 1
        end
        if size(A) != (r,r)
            println("A should be $r x $r square matrix.")
            counter = counter + 1
        end
        if size(Q) != (r,r)
            println("Q should be $r x $r square matrix.")
            counter = counter + 1
        end
    end
    
    if n == 1
        if typeof(R) != Array{Float64,1} && typeof(R) != Array{Int64,1}
            println("R has to be 1x1 matrix.")
            counter = counter + 1
        end
    else
        if typeof(R) != Array{Float64,2} && typeof(R) != Array{Int64,2}
            println("R has to be a matrix.")
            counter = counter + 1
        end
        if size(R) != (n,n)
            println("R should be $n x $n square matrix.")
            counter = counter + 1
        end
    end
    
    if size(E,1) != n
        println("E should have $r rows.")
        counter = counter + 1
    end
    
    ### Checking the dimension of dataset
    if (size(y,1) != n)
        println("observation should be $r x 1 matrix.")
        counter = counter + 1
    end 
    
        
    ### Result of dianostic
    if counter == 0
        return "We set up correctly!"
    end
    return "the setup should be as specified above."
end

check_kalman (generic function with 1 method)

In [2]:
datra = CSV.read("epic_matrix_quarterly.csv")

,date,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPL,MRTSSM44X72USS
,Date⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,1959-01-01,0.00393351,0.00357626,0.0103497,missing,missing
2,1959-02-01,0.00643111,0.00737371,0.00939402,missing,missing
3,1959-03-01,0.00649814,0.00701939,-0.0035764,missing,missing
4,1959-04-01,0.00582628,0.00662948,0.0119843,missing,missing
5,1959-05-01,0.003108,0.00302211,0.00364585,missing,missing
6,1959-06-01,-0.00058554,-0.00080784,-0.00336493,missing,missing
7,1959-07-01,-0.00569527,-0.00571601,0.0059929,missing,missing
8,1959-08-01,0.000788317,-4.27798e-5,0.0100012,missing,missing
9,1959-09-01,0.00122307,0.00119714,-0.00682331,missing,missing


### 1. demeaning observation

In [3]:
for i in 2:size(datra,2)
    datra[i] = replace(datra[i], NaN => missing) # handle NaN and missing
    temp_mean = mean(skipmissing(datra[i]))
    println(temp_mean)
    for j in 1:size(datra,1)
        temp = datra[j,i] - temp_mean
        datra[i] = replace(datra[i], datra[j,i] => temp)
    end
end
datra

0.002680903028916844
0.002510959041484842
0.002679198788509781
0.002172054491574117
0.003505434687908145
0.002174513917737261
0.002062279626846661
0.002122725235089939
0.0016272045515276226
0.002353644156078295
0.001364491930126037
0.003509447452772167
0.0023294956406565187
0.002956389376440745
0.0016727666104089784
0.0022062231033191607
0.0023368939586762578
0.0014645458860395916
-0.006585497237569057
0.0012058148817851286
0.0012401723223976011
-0.0033149171270718232
0.010773480662983422
0.0004288002332892715
0.0003976462811957301
0.0005620120822922685
0.000409505008844792
0.0006650155930904974
0.0034508300327457577
0.0014604637414962912
0.00015820209573804073
-3.879821285988023e-6
0.0012656872268337828
-0.0002150043309828543
-0.00011221657858879582
-0.00037270524356778
0.0018653975182767494
0.0013096021592589268
0.0011567690490889586
0.0014934545524258782
0.0017614806214247593
0.0014110046204204338
40.26491712707177
0.002348066298342542
40.758149171270745
7.222039890478135
5.07873573

,date,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPL,MRTSSM44X72USS
,Date⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,1959-01-01,0.0012526,0.0010653,0.00767048,missing,missing
2,1959-02-01,0.0037502,0.00486275,0.00671482,missing,missing
3,1959-03-01,0.00381723,0.00450843,-0.0062556,missing,missing
4,1959-04-01,0.00314537,0.00411852,0.00930512,missing,missing
5,1959-05-01,0.000427094,0.000511156,0.000966653,missing,missing
6,1959-06-01,-0.00326644,-0.0033188,-0.00604413,missing,missing
7,1959-07-01,-0.00837617,-0.00822697,0.00331371,missing,missing
8,1959-08-01,-0.00189259,-0.00255374,0.007322,missing,missing
9,1959-09-01,-0.00145783,-0.00131381,-0.00950251,missing,missing


In [4]:
CSV.write("demeaned_matrix.csv", datra, writeheader = true)

"demeaned_matrix.csv"

### 2. Find the largest chunk without missing value.

In [5]:
### This matrix has been manually edited in excel. I remove some series that have weird structures and lots of missing values.
data = CSV.read("demeaned_matrix_prior.csv")

,date,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPL,MRTSSM44X72USS
,String,Float64,Float64,Float64,Float64⍰,Float64⍰
1,1/1/59,0.0012526,0.0010653,0.00767048,missing,missing
2,2/1/59,0.0037502,0.00486275,0.00671482,missing,missing
3,3/1/59,0.00381723,0.00450843,-0.0062556,missing,missing
4,4/1/59,0.00314537,0.00411852,0.00930512,missing,missing
5,5/1/59,0.000427094,0.000511156,0.000966653,missing,missing
6,6/1/59,-0.00326644,-0.0033188,-0.00604413,missing,missing
7,7/1/59,-0.00837617,-0.00822697,0.00331371,missing,missing
8,8/1/59,-0.00189259,-0.00255374,0.007322,missing,missing
9,9/1/59,-0.00145783,-0.00131382,-0.00950251,missing,missing


In [6]:
non_missing_index = collect(1:725)
for i in 1:size(data,2)
    non_missing_index[findall(ismissing, data[i])].= 0
end

print(non_missing_index)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [7]:
for i in 396:724
    if non_missing_index[i] == 0
        println(i, "zero!!!")
    end
end

### row 398 to 723 is the largest component



396zero!!!
397zero!!!
724zero!!!


In [223]:
prior_df = data[398:723,2:end]

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPL,MRTSSM44X72USS,INDPRO
,Float64,Float64,Float64,Float64⍰,Float64⍰,Float64
1,-0.00102306,-0.00169723,-0.00085829,0.00184479,-0.00650604,0.00610109
2,0.00103965,0.000230497,-0.00184127,0.00446995,0.00251109,0.0053776
3,0.00265047,0.00299273,0.00247554,-0.0109105,0.00196199,0.00108208
4,0.0014802,0.00193052,1.2e-5,0.00831459,-0.000701602,-0.00203149
5,-0.00347101,-0.00386,0.00065705,0.0111309,0.00357458,0.00682233
6,0.00118623,0.00130185,-0.00019748,-0.0182077,-0.000264192,-0.00710763
7,-0.00650398,-0.00747332,0.00380398,0.0116358,0.00598455,0.000146118
8,-0.00761176,-0.00830757,0.000685958,0.00186721,0.00369995,0.00529834
9,-0.000498748,0.00196203,-0.000974791,-0.000469345,-0.0012925,0.00207053


In [9]:
prior_matrix = convert(Array, prior_df)

┌ Warning: `convert(::Type{Array}, df::AbstractDataFrame)` is deprecated, use `convert(Matrix, df)` instead.
│   caller = top-level scope at In[9]:1
└ @ Core In[9]:1


326×118 Array{Union{Missing, Float64},2}:
 -0.00102306   -0.00169723   -0.00085829   …  -0.00584718    0.000665271
  0.00103965    0.000230497  -0.00184127       0.000202901   0.00462347 
  0.00265047    0.00299273    0.00247554       0.00880858   -0.00127922 
  0.0014802     0.00193052    1.2e-5          -0.00854766    0.00252399 
 -0.00347101   -0.00386       0.00065705       0.0116299     0.000467616
  0.00118623    0.00130185   -0.00019748   …  -0.00736782   -0.00846585 
 -0.00650398   -0.00747332    0.00380398      -0.00744271    0.00153729 
 -0.00761176   -0.00830757    0.000685958      0.0120916     0.00282071 
 -0.000498748   0.00196203   -0.000974791     -0.00302042   -0.000354051
  0.0331231     0.0376036     0.00390239      -0.0143728    -0.0113346  
 -0.0305959    -0.0376566    -0.00413035   …   0.00713867    0.012571   
  0.00190246    0.0030889    -0.000831073     -0.00129334    0.00505875 
 -0.00529141   -0.00637894   -0.0053748        0.00795123   -0.00560051 
  ⋮      

In [10]:
CSV.write("prior_matrix.csv", prior_df, writeheader = false)

"prior_matrix.csv"

### 3. Principal component

In [57]:
using RDatasets, MultivariateStats

prior_PCA = CSV.read("prior_matrix.csv", header = false)
prior_PCA = convert(Array, prior_PCA)

┌ Warning: `convert(::Type{Array}, df::AbstractDataFrame)` is deprecated, use `convert(Matrix, df)` instead.
│   caller = top-level scope at In[57]:4
└ @ Core In[57]:4


326×118 Array{Float64,2}:
 -0.00102306   -0.00169723   -0.00085829   …  -0.00584718    0.000665271
  0.00103965    0.000230497  -0.00184127       0.000202901   0.00462347 
  0.00265047    0.00299273    0.00247554       0.00880858   -0.00127922 
  0.0014802     0.00193052    1.2e-5          -0.00854766    0.00252399 
 -0.00347101   -0.00386       0.00065705       0.0116299     0.000467616
  0.00118623    0.00130185   -0.00019748   …  -0.00736782   -0.00846585 
 -0.00650398   -0.00747332    0.00380398      -0.00744271    0.00153729 
 -0.00761176   -0.00830757    0.000685958      0.0120916     0.00282071 
 -0.000498748   0.00196203   -0.000974791     -0.00302042   -0.000354051
  0.0331231     0.0376036     0.00390239      -0.0143728    -0.0113346  
 -0.0305959    -0.0376566    -0.00413035   …   0.00713867    0.012571   
  0.00190246    0.0030889    -0.000831073     -0.00129334    0.00505875 
 -0.00529141   -0.00637894   -0.0053748        0.00795123   -0.00560051 
  ⋮                      

In [58]:
M = fit(PCA, prior_PCA, maxoutdim = 326, pratio = 1.0)

PCA(indim = 326, outdim = 118, principalratio = 1.00000)

labor: real associated with market
nominal: price
real: quantity

survey: survey

output pca: actual component, and the remainder matrix.
- 

In [59]:
y = transform(M, prior_PCA)
#reconstruct(M, y)[:,1]

118×118 Array{Float64,2}:
   -1.62095e5      -1.62095e5    …    -1.62095e5      -1.62095e5  
 -384.21         -384.21            -384.216        -384.214      
   -2.71121        -2.70194           -2.66597        -2.70863    
   -8.00722        -8.00494           -7.97494        -8.01179    
   -2.37851        -2.38103           -2.4021         -2.34841    
    0.797908        0.799465     …     0.774329        0.799578   
   -0.175178       -0.177506          -0.144432       -0.172536   
    0.290388        0.291273           0.312757        0.278015   
    0.154221        0.156373           0.10337         0.162376   
   -0.820866       -0.818778          -0.828527       -0.825093   
    0.353082        0.351388     …     0.340854        0.333287   
   -1.13331        -1.13695           -1.17108        -1.1394     
    0.0519798       0.0531303          0.0640827       0.0371295  
    ⋮                            ⋱                                
   -7.48221e-5      0.000101485     

In [60]:
p = projection(M)

326×118 Array{Float64,2}:
 -0.0154655  -0.0888199     0.0094992   …  -0.0805664    -0.365492  
 -0.0154109  -0.0888152     0.00949378     -0.0421734    -0.018819  
 -0.0157155  -0.0597371    -0.00924236     -0.0345907     0.0532612 
 -0.0158028  -0.0559008    -0.0287703      -0.134118      0.00761778
 -0.0157714  -0.0734852     0.100095       -0.0324559     0.0216346 
 -0.0157578  -0.0734749     0.100101    …  -0.0629455     0.0626036 
 -0.0157387  -0.0734749     0.100093       -0.0368075    -0.0301859 
 -0.0156677  -0.0946373     0.0388241       0.0149188    -0.00326446
 -0.0155774  -0.0734405     0.100108        0.0152083     0.0877049 
 -0.0154941  -0.07705       0.0788023       0.0145309     0.0108076 
 -0.0153965  -0.0770391     0.0788157   …   0.0459198     0.0308553 
 -0.0154279  -0.0632328    -0.0407133      -0.00557542    0.0163395 
 -0.0155338  -0.064221     -0.0358478      -0.000105982  -0.0150868 
  ⋮                                     ⋱                           
  0.0940

In [15]:
x_reduced = p[:,2:end] * y[2:end,:]

326×118 Array{Float64,2}:
 33.8122    33.8116    33.8123    …  33.8504    33.8074    33.8139  
 33.3275    33.3267    33.3246       33.2735    33.3267    33.3311  
 23.1378    23.1382    23.1376       23.1808    23.144     23.1339  
 21.6991    21.6996    21.6976       21.6941    21.6891    21.7002  
 27.9424    27.942     27.9465       27.9395    27.9575    27.9463  
 28.411     28.4112    28.4096    …  28.4011    28.4025    28.4014  
 27.7871    27.7862    27.7973       27.7944    27.7862    27.7951  
 35.7689    35.7683    35.7772       35.7706    35.7887    35.7794  
 28.2372    28.2397    28.2367       28.2412    28.2347    28.2373  
 29.2323    29.2368    29.203        29.2027    29.1848    29.1879  
 29.2553    29.2483    29.2817    …  29.2404    29.293     29.2985  
 24.8132    24.8145    24.8104       24.8533    24.8101    24.8164  
 24.9711    24.97      24.9709       24.9945    24.9843    24.9708  
  ⋮                               ⋱   ⋮                             
 -5.2800

In [68]:
f_G = p[:,1]

326-element Array{Float64,1}:
 -0.015465542556305037
 -0.015410852487686543
 -0.01571554710647047 
 -0.01580279239427524 
 -0.01577136442329982 
 -0.0157577527453626  
 -0.01573865804895142 
 -0.015667733406371488
 -0.015577362553049765
 -0.015494061504209986
 -0.015396488802840964
 -0.015427874719542505
 -0.015533847606805113
  ⋮                   
  0.09406102761399797 
  0.09069006013059379 
  0.0889194476003686  
  0.08692221844824095 
  0.08490741500597815 
  0.08291567102696273 
  0.08071433417035961 
  0.07784771857163104 
  0.07361683301787773 
  0.07087062103444455 
  0.07120744744176233 
  0.07184800307788425 

In [48]:
println(names(datra)[2:end])
println()
println(names(data)[2:end])

Symbol[:RPI, :W875RX1, :DPCERA3M086SBEA, :CMRMTSPL, :MRTSSM44X72USS, :INDPRO, :IPFPNSS, :IPFINAL, :IPCONGD, :IPDCONGD, :IPNCONGD, :IPBUSEQ, :IPMAT, :IPDMAT, :IPNMAT, :IPMANSICS, :IPB51222S, :IPFUELS, :CUMFNS, :CLF16OV, :CE16OV, :UNRATE, :UEMPMEAN, :UEMPLT5, :UEMP5TO14, :UEMP15OV, :UEMP15T26, :UEMP27OV, :ICSA, :PAYEMS, :USGOOD, :CES1021000001, :USCONS, :MANEMP, :DMANEMP, :NDMANEMP, :SRVPRD, :USTPU, :USWTRADE, :USTRADE, :USFIRE, :USGOVT, :CES0600000007, :AWOTMAN, :AWHMAN, :HOUST, :HOUSTNE, :HOUSTMW, :HOUSTS, :HOUSTW, :PERMIT, :PERMITNE, :PERMITMW, :PERMITS, :PERMITW, :ACOGNO, :DGORDER, :NEWORDER, :AMDMUO, :BUSINV, :ISRATIO, :M1SL, :M2SL, :M2REAL, :AMBSL, :TOTRESNS, :NONBORRES, :BUSLOANS, :REALLN, :NONREVSL, :PI, :SP500, :DJIA, :M1346BUSM156NNBR, :FEDFUNDS, :CP3M, :TB3MS, :TB6MS, :GS1, :GS5, :GS10, :AAA, :BAA, :CPFF, :TB3SMFFM, :TB6SMFFM, :T1YFFM, :T5YFFM, :T10YFFM, :AAAFFM, :BAAFFM, :TWEXMMTH, :EXSZUS, :EXJPUS, :EXUSUK, :EXCAUS, :WPSFD49207, :WPSFD49502, :WPSID61, :WPSID62, :WTISPLC, :PP

In [49]:
length(names(data)) 
## remove 9 series including ?both GDPC1, GDPCTPI, ICSA, VXOCLS, :SP500, :DJIA, :M1346BUSM156NNBR, CPFF, CP3M 

119

In [81]:
R = [:RPI,:W875RX1,:DPCERA3M086SBEA,:CMRMTSPL,:INDPRO,:IPFPNSS,:IPFINAL,:IPCONGD,:IPDCONGD,:IPNCONGD,:IPBUSEQ,:IPMAT,:IPDMAT,:IPNMAT,:IPMANSICS,:IPB51222S,:IPFUELS
    ,:CUMFNS,:ISRATIO,:M2REAL,:HOUST,:HOUSTNE,:HOUSTMW,:HOUSTS,:HOUSTW,:PERMIT,:PERMITNE,:PERMITMW,:PERMITS,:PERMITW
    ,:PI,:WPSFD49207,:WPSFD49502,:WPSID61,:WPSID62,:WTISPLC,:PPICMM,:CPIAUCSL,:CPIAPPSL,:CPITRNSL,:CPIMEDSL,:CUSR0000SAC
    ,:CUSR0000SAD,:CUSR0000SAS,:CPIULFSL,:CUSR0000SA0L2,:CUSR0000SA0L5,:GDPC1,:GDPCTPI]
S = [:UMCSENT,:DTCOLNVHFNM,:DTCTHFNM]
L = [:CLF16OV,:CE16OV,:UNRATE,:UEMPMEAN,:UEMPLT5,:UEMP5TO14,:UEMP15OV,:UEMP15T26,:UEMP27OV,:ICSA,:PAYEMS,:USGOOD,:CES1021000001,:USCONS,:MANEMP,:DMANEMP,:NDMANEMP,:SRVPRD,:USTPU,:USWTRADE,:USTRADE,:USFIRE,:USGOVT,:CES0600000007,:AWOTMAN,:AWHMAN,:CES0600000008,:CES2000000008,:CES3000000008]
# otherwise, assign to nominal factor


29-element Array{Symbol,1}:
 :CLF16OV      
 :CE16OV       
 :UNRATE       
 :UEMPMEAN     
 :UEMPLT5      
 :UEMP5TO14    
 :UEMP15OV     
 :UEMP15T26    
 :UEMP27OV     
 :ICSA         
 :PAYEMS       
 :USGOOD       
 :CES1021000001
 ⋮             
 :SRVPRD       
 :USTPU        
 :USWTRADE     
 :USTRADE      
 :USFIRE       
 :USGOVT       
 :CES0600000007
 :AWOTMAN      
 :AWHMAN       
 :CES0600000008
 :CES2000000008
 :CES3000000008

In [38]:
prior_colname = names(data)[2:end]


118-element Array{Symbol,1}:
 :RPI            
 :W875RX1        
 :DPCERA3M086SBEA
 :CMRMTSPL       
 :MRTSSM44X72USS 
 :INDPRO         
 :IPFPNSS        
 :IPFINAL        
 :IPCONGD        
 :IPDCONGD       
 :IPNCONGD       
 :IPBUSEQ        
 :IPMAT          
 ⋮               
 :PCEPI          
 :DDURRG3M086SBEA
 :DNDGRG3M086SBEA
 :DSERRG3M086SBEA
 :CES0600000008  
 :CES2000000008  
 :CES3000000008  
 :UMCSENT        
 :MZMSL          
 :DTCOLNVHFNM    
 :DTCTHFNM       
 :INVEST         

In [85]:
prior_colname_ind = Array{String}(undef, length(prior_colname))
for i in 1:length(prior_colname)
    counter = 0
    for a in 1:length(R)
        if R[a] == prior_colname[i]
            prior_colname_ind[i] = "R"
            counter = counter + 1
        break
        end
    end
    
    for b in 1:length(S)
        if S[b] == prior_colname[i]
            prior_colname_ind[i] = "S"
            counter = counter +1
        break
        end
    end
    
    for c in 1:length(L)
        if L[c] == prior_colname[i]
            prior_colname_ind[i] = "L"
            counter = counter + 1
        break
        end
    end
    
    if counter == 0
        prior_colname_ind[i] = "N"
    end
end
        

In [194]:
print(prior_colname_ind)

["R", "R", "R", "R", "N", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "N", "N", "N", "N", "N", "R", "N", "N", "R", "N", "N", "N", "N", "N", "N", "R", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "N", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "R", "N", "N", "N", "N", "L", "L", "L", "S", "N", "S", "S", "N"]

In [96]:
R_col = findall(x -> x == "R", prior_colname_ind)
S_col= findall(x -> x == "S", prior_colname_ind)
L_col = findall(x -> x == "L", prior_colname_ind)
N_col = findall(x -> x == "N", prior_colname_ind)
println()

In [91]:
x_reduced_R = x_reduced[:,R_col]
M_R = fit(PCA, x_reduced_R, maxoutdim = 326, pratio = 1.0)
f_R = projection(M_R)[:,1]

x_reduced_N = x_reduced[:,N_col]
M_N = fit(PCA, x_reduced_N, maxoutdim = 326, pratio = 1.0)
f_N = projection(M_N)[:,1]

x_reduced_L = x_reduced[:,L_col]
M_L = fit(PCA, x_reduced_L, maxoutdim = 326, pratio = 1.0)
f_L = projection(M_L)[:,1]

x_reduced_S = x_reduced[:,S_col]
M_S = fit(PCA, x_reduced_S, maxoutdim = 326, pratio = 1.0)
f_S = projection(M_S)[:,1]

326-element Array{Float64,1}:
 -0.08979379199721609 
 -0.043906946034085244
 -0.07041415432005703 
 -0.05895539522719852 
 -0.0697597058009971  
 -0.10489866716823577 
 -0.03820232753471525 
 -0.06771960239465903 
 -0.07661397617634151 
 -0.04642296627637935 
 -0.07880283004599435 
 -0.0770822877714244  
 -0.07657479526097682 
  ⋮                   
 -0.02814207315035285 
 -0.04214121608031785 
 -0.03468282840024315 
 -0.016539584194207573
 -0.01797264833848361 
 -0.04623594370648884 
 -0.07588521235411436 
 -0.06826911571861484 
 -0.04041808795551904 
 -0.056330689151963556
 -0.04858874970538942 
 -0.00958957779133858 

In [98]:
hcat(f_G,f_L,f_S,f_N,f_R)

326×5 Array{Float64,2}:
 -0.0154655  -0.0888093    -0.103543    -0.0854459    -0.0897938 
 -0.0154109  -0.0887995    -0.0148086   -0.0834812    -0.0439069 
 -0.0157155  -0.0597353    -0.0260516   -0.0581564    -0.0704142 
 -0.0158028  -0.0558929    -0.0145268   -0.0552562    -0.0589554 
 -0.0157714  -0.0735222     0.0548368   -0.0735859    -0.0697597 
 -0.0157578  -0.0735193     0.00947875  -0.0742388    -0.104899  
 -0.0157387  -0.0735182     0.00954753  -0.0705146    -0.0382023 
 -0.0156677  -0.0946391     0.0360647   -0.089197     -0.0677196 
 -0.0155774  -0.0734869    -0.172965    -0.071115     -0.076614  
 -0.0154941  -0.0770832    -0.08175     -0.0740812    -0.046423  
 -0.0153965  -0.0770623     0.0271019   -0.0738203    -0.0788028 
 -0.0154279  -0.0632254     0.0411393   -0.0650615    -0.0770823 
 -0.0155338  -0.0642065     0.0103039   -0.066878     -0.0765748 
  ⋮                                                              
  0.094061    0.0165433     0.0169677    0.013672   

### 4. finding all estimates (of prior)

In [307]:
CSV.write("prior_matrix_est.csv", prior_df, writeheader = true)
prior_df_est = CSV.read("prior_matrix_est.csv", header = true)

,f_G_lag,f_G,f_S_lag,f_S,f_L_lag,f_L,f_N_lag
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.0154109,-0.0154655,-0.0148086,-0.103543,-0.0887995,-0.0888093,-0.0834812
2,-0.0157155,-0.0154109,-0.0260516,-0.0148086,-0.0597353,-0.0887995,-0.0581564
3,-0.0158028,-0.0157155,-0.0145268,-0.0260516,-0.0558929,-0.0597353,-0.0552562
4,-0.0157714,-0.0158028,0.0548368,-0.0145268,-0.0735222,-0.0558929,-0.0735859
5,-0.0157578,-0.0157714,0.00947875,0.0548368,-0.0735193,-0.0735222,-0.0742388
6,-0.0157387,-0.0157578,0.00954753,0.00947875,-0.0735182,-0.0735193,-0.0705146
7,-0.0156677,-0.0157387,0.0360647,0.00954753,-0.0946391,-0.0735182,-0.089197
8,-0.0155774,-0.0156677,-0.172965,0.0360647,-0.0734869,-0.0946391,-0.071115
9,-0.0154941,-0.0155774,-0.08175,-0.172965,-0.0770832,-0.0734869,-0.0740812


In [224]:
using Distributions, GLM

f_R_lag = append!(f_R[2:end],-9999)
f_N_lag = append!(f_N[2:end],-9999)
f_L_lag = append!(f_L[2:end],-9999)
f_S_lag = append!(f_S[2:end],-9999)
f_G_lag = append!(f_G[2:end],-9999)

insertcols!(prior_df_est,1,:f_R => f_R)
insertcols!(prior_df_est,1,:f_R_lag => f_R_lag)
insertcols!(prior_df_est,1,:f_N => f_N)
insertcols!(prior_df_est,1,:f_N_lag => f_N_lag)
insertcols!(prior_df_est,1,:f_L => f_L)
insertcols!(prior_df_est,1,:f_L_lag => f_L_lag)
insertcols!(prior_df_est,1,:f_S => f_S)
insertcols!(prior_df_est,1,:f_S_lag => f_S_lag)
insertcols!(prior_df_est,1,:f_G => f_G)
insertcols!(prior_df_est,1,:f_G_lag => f_G_lag)
insertcols!(prior_df_est,1, :coef => ones(326))

,f_G_lag,f_G,f_S_lag,f_S,f_L_lag,f_L,f_N_lag
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.0154109,-0.0154655,-0.0148086,-0.103543,-0.0887995,-0.0888093,-0.0834812
2,-0.0157155,-0.0154109,-0.0260516,-0.0148086,-0.0597353,-0.0887995,-0.0581564
3,-0.0158028,-0.0157155,-0.0145268,-0.0260516,-0.0558929,-0.0597353,-0.0552562
4,-0.0157714,-0.0158028,0.0548368,-0.0145268,-0.0735222,-0.0558929,-0.0735859
5,-0.0157578,-0.0157714,0.00947875,0.0548368,-0.0735193,-0.0735222,-0.0742388
6,-0.0157387,-0.0157578,0.00954753,0.00947875,-0.0735182,-0.0735193,-0.0705146
7,-0.0156677,-0.0157387,0.0360647,0.00954753,-0.0946391,-0.0735182,-0.089197
8,-0.0155774,-0.0156677,-0.172965,0.0360647,-0.0734869,-0.0946391,-0.071115
9,-0.0154941,-0.0155774,-0.08175,-0.172965,-0.0770832,-0.0734869,-0.0740812


In [325]:
prior_df_est[120:end]

,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENT
,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.00214931,0.000808085,0.000833808,-0.00583056,0.00176137,7.16714
2,0.000981954,-0.00259793,-0.00251064,-0.00072134,-0.000887602,0.96996
3,0.00319504,0.000269384,0.00250402,0.00796178,-0.000880603,1.80282
4,-0.00232729,0.00262281,-0.00416543,-0.0151822,-6.98e-7,0.96996
5,-0.000315732,-0.0028357,0.00416445,0.0115805,0.000872294,-3.83286
6,-0.00134002,0.001313,-0.00498663,-0.0108637,-0.00349184,-0.697177
7,0.000202228,0.00239379,0.00498845,0.0137757,0.00262056,-0.697177
8,-0.000220322,-0.00236479,-0.00166159,-0.00507755,-6.66e-7,-2.53004
9,0.0017979,-2.44e-5,-0.000825857,-0.000718697,-6.56e-7,11.9671


In [371]:
arr_μ = Array{Float64}(undef, 118)
arr_Λ_G = Array{Float64}(undef, 118)
arr_Λ_S = Array{Float64}(undef, 118)
arr_Λ_L = Array{Float64}(undef, 118)
arr_Λ_N = Array{Float64}(undef, 118)
arr_Λ_R = Array{Float64}(undef, 118)
arr_ρ = Array{Float64}(undef, 118)
arr_σ2_ie = Array{Float64}(undef, 118)

for i in 12:size(prior_df_est,2)
    idx = prior_colname_ind[i-11]
    title = prior_colname[i-11]
    
    if idx == "S"
        fit_lm = lm(convert(Matrix,prior_df_est[:,[1,3,5]]) , prior_df_est[:,i])
        arr_μ[i-11], arr_Λ_G[i-11], arr_Λ_S[i-11], arr_Λ_L[i-11], arr_Λ_N[i-11], arr_Λ_R[i-11] =
            coef(fit_lm)[1], coef(fit_lm)[2], coef(fit_lm)[3], 0, 0, 0
    elseif idx == "L"
        fit_lm = lm(convert(Matrix,prior_df_est[:,[1,3,7]]) , prior_df_est[:,i])
        arr_μ[i-11], arr_Λ_G[i-11], arr_Λ_S[i-11], arr_Λ_L[i-11], arr_Λ_N[i-11], arr_Λ_R[i-11] =
            coef(fit_lm)[1], coef(fit_lm)[2], 0, coef(fit_lm)[3], 0, 0
    elseif idx == "N"
        fit_lm = lm(convert(Matrix,prior_df_est[:,[1,3,9]]) , prior_df_est[:,i])
        arr_μ[i-11], arr_Λ_G[i-11], arr_Λ_S[i-11], arr_Λ_L[i-11], arr_Λ_N[i-11], arr_Λ_R[i-11] =
            coef(fit_lm)[1], coef(fit_lm)[2], 0, 0, coef(fit_lm)[3], 0
    else
        fit_lm = lm(convert(Matrix,prior_df_est[:,[1,3,11]]) , prior_df_est[:,i])
        arr_μ[i-11], arr_Λ_G[i-11], arr_Λ_S[i-11], arr_Λ_L[i-11], arr_Λ_N[i-11], arr_Λ_R[i-11] =
            coef(fit_lm)[1], coef(fit_lm)[2], 0, 0, 0, coef(fit_lm)[3]
    end

    resid_lm = lm(hcat(ones(size(residuals(fit_lm),1)-1), residuals(fit_lm)[2:end]), residuals(fit_lm)[1:end-1])
    arr_ρ[i-11] = coef(resid_lm)[2]
    
    ## Demeaned residuals?
    resid_mean = mean(residuals(fit_lm))
    demeaned_resid = residuals(fit_lm)
    for i in 1:length(demeaned_resid)
        demeaned_resid[i] = demeaned_resid[i] - resid_mean
    end
    arr_σ2_ie[i-11] = var(demeaned_resid)
end

arr_β = Array{Float64}(undef, 5)
arr_σ2_ju = Array{Float64}(undef, 5)

for i in 1:5
    fit_lm = lm(convert(Matrix, prior_df_est[:,[1,2*i]]), prior_df_est[:,2*i+1])
    arr_β[i] = coef(fit_lm)[2]
    
    ## Demeaned residuals?
    demeaned_resid = residuals(fit_lm)
     resid_mean = mean(demeaned_resid)
    for i in 1:length(demeaned_resid)
        demeaned_resid[i] = demeaned_resid[i] - resid_mean
    end
    arr_σ2_ju = var(demeaned_resid)
end

In [377]:
### Explore the prior parameters' estimates.
#arr_Λ_N
#coef(lm_temp)
#stderror(lm_temp)
#residuals(lm_temp)

#arr_μ
#arr_Λ_G
#arr_Λ_S
#arr_Λ_L
#arr_ρ
#arr_σ2_ie

arr_β
#arr_σ2_ju

118-element Array{Float64,1}:
  0.00605120069793919   
  0.007498097451806681  
  0.0037195756744128243 
  0.003829461949448205  
 -0.015458711543422913  
  0.003951176186603654  
  0.0015920882243201423 
 -0.0038630232473124667 
 -0.00041154272123505175
 -0.0013023911376056942 
  0.0014221109362633866 
 -0.010470855453743202  
  0.00606121031303879   
  ⋮                     
 -0.0032739708131232963 
 -0.0007156274465771473 
 -0.00751190949293436   
 -0.002429557081860376  
  4.6585940229454734e-5 
 -0.0011779802176826303 
  0.0018434100601861915 
  2.0052161839051377    
  0.014780916350812454  
 -0.0012326473328971848 
  0.0011328475022756863 
 -0.0075881119500074866 

In [384]:
### Creating state-evolutionary matrix [Still incomplete!]
hcat(arr_Λ_G, arr_Λ_S, arr_Λ_L, arr_Λ_N, arr_Λ_R, zeros(118,5), zeros(118,5), zeros(118,5), zeros(118,5), ones(118), zeros(118), zeros(118), zeros(118), zeros(118), zeros(118))


118×31 Array{Float64,2}:
  0.0060512      0.0         …  0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.0074981      0.0            0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.00371958     0.0            0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.00382946     0.0            0.0  1.0  0.0  0.0  0.0  0.0  0.0
 -0.0154587      0.0            0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.00395118     0.0         …  0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.00159209     0.0            0.0  1.0  0.0  0.0  0.0  0.0  0.0
 -0.00386302     0.0            0.0  1.0  0.0  0.0  0.0  0.0  0.0
 -0.000411543    0.0            0.0  1.0  0.0  0.0  0.0  0.0  0.0
 -0.00130239     0.0            0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.00142211     0.0         …  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 -0.0104709      0.0            0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.00606121     0.0            0.0  1.0  0.0  0.0  0.0  0.0  0.0
  ⋮                          ⋱       ⋮                        ⋮  
 -0.00327397     0.0            0.0  1.0  0.0  0.0 

In [ ]:
# y_t - μ = Z_θ * x_t
# x_{t+1} = A_θ * x_t + η_t

#arr_μ

Z_θ = hcat(arr_Λ_G, arr_Λ_S, arr_Λ_L, arr_Λ_N, arr_Λ_R, zeros(118,5), zeros(118,5), zeros(118,5), zeros(118,5), ones(118), zeros(118), zeros(118), zeros(118), zeros(118), zeros(118))


In [383]:
zeros(118,5)

118×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 ⋮                      
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

### 5. Expectation Maximization

In [ ]:
max_iter = 5000 # EM loop maximum number of iterations.

while num_iter < max_iter & ~converged
    [] = Emstep(...) # Applying EM algorithm.

In [ ]:
function EMstep(y,A,G,Q,R,x_0, σ_0, r, p = 1, R_mat,q, nQ, i_idio, blocks)
    
    (n,T) = size(y)
    nM = n - nQ
    #pC = size(R_mat,2)
    #ppc = max(p, pC)
    num_blocks = size(blocks, 2) # how many columns are there?
    
    [x_sm, σ_sm, vv_sm, loglik] = runKF(y, A, C, Q, R, x_0, σ_0)
    
    A_new = A
    Q_new = Q
    σ_0_new = σ_0
    
    ##### Transition equation
    ### Update factor parameters individually
    for i = 1:num_blocks
        r_i = r[i]
        rp = r_i*p
        rp1 = sum(r[1:i-1])*ppC
        b_subset = rp1+1:rp1+rp # Subset blocks: Helps for subsetting x_sm, σ_sm
        t_start = rp1+1
        t_end = rp1+r_i*ppC
        
        EZZ = x_sm[b_subset, 2:end]*(x_sm[b_subset, 2:end])' + sum(σ_sm[b_subset, b_subset, 2:end], 3)
        EZZ_BB = x_sm[b_subset, 1:end-1]*(x_sm[b_subset, 1:end-1])' + sum(σ_sm[b_subset, b_subset, 1:end-1], 3)
        EZZ_FB = x_sm[b_subset, 2:end]*(x_sm[b_subset, 1:end-1])' + sum(vv_sm[b_subset, b_subset, :], 3)
        
        A_i = A[t_start:t_end, t_start:t_end]
        Q_i = Q[t_start:t_end, t_start:t_end]
        
        A_i[1:r_i,1:rp] = EZZ_FB[1:r_i,1:rp]*inv(EZZ_BB[1:r_i,1:rp])
        Q_i[1:r_i,1:r_i] = (EZZ[1:r_i,1:r_i] - A_i[1:r_i,1:rp]*EZZ_FB[1:r_i,1:rp]') / T
        
        A_new[t_start:t_end, t_start:t_end] = A_i
        Q_new[t_start:t_end, t_start:t_end] = Q_i
        σ_0_new[t_start:t_end, t_start:t_end] = σ_sm[t_start:t_end, t_start:t_end, 1]
    end
    
    ### Update parameters for idiosyncratic component
    rp1 = sum(r)*ppC
    niM = sum(i_idio[1:nM])
    t_start = rp1+1
    i_subset = t_start:rp1+niM
    
    EZZ = Diagonal(Diagonal(x_sm[t_start:end,2:end]*(x_sm[t_start:end, 2:end])'))+Diagonal(Diagonal(sum(σ_sm[t_start:end, t_start:end, 2:end], 3)))
    EZZ_BB = Diagonal(Diagonal(x_sm[t_start:end,1:end-1]*(x_sm[t_start:end, 1:end-1])'))+Diagonal(Diagonal(sum(σ_sm[t_start:end, t_start:end, 1:end-1], 3)))
    EZZ_FB = Diagonal(Diagonal(x_sm[t_start:end,2:end]*(x_sm[t_start:end, 1:end-1])'))+Diagonal(Diagonal(sum(vv_sm[t_start:end, t_start:end, :], 3)))
    
    A_i = EZZ_FB*Diagonal(1./Diagonal((EZZ_BB)))
    Q_i = (EZZ - A_i*EZZ_FB') / T
    
    A_new[i_subset, i_subset] = A_i[1:niM,1:niM]
    Q_new[i_subset, i_subset] = Q_i[1:niM,1:niM]
    V_0_new[i_subset, i_subset] = Diagonal(Diagonal(σ_sm[i_subset,i_subset,1]))
    
    ##### Observation equation
    ### Maximization step
    
    
    
    
    
    return G_new, R_new, A_new, Q_new, x_0, σ_0, loglik
end

In [10]:
mat = [1 2 3
2 3 4]

mat[1:2,1:2] = [0 0
              0 0]

mat


2×3 Array{Int64,2}:
 0  0  3
 0  0  4

In [16]:
using LinearAlgebra, Statistics, Compat
Diagonal([1, 2, 3])

3×3 Diagonal{Int64,Array{Int64,1}}:
 1  ⋅  ⋅
 ⋅  2  ⋅
 ⋅  ⋅  3

### 6. Filtering???

In [ ]:
A = [ 0.9 0.09
      0.5 0.5 ]
Q = [ 0.0125 0.0285
      0.0285 0.0909 ]
G = [ 1.0000 -0.5
      0       1.0
      0.25    0.25 ]
E = [ -1.3143 0.4297 -1.1545 -1.2590
      -1.1900 -0.0553 -0.8746 -0.6032 
      0.5864 -1.4580 0.0189 -0.3158 ]
R = [ 0.0625 0 0
      0 0.8100 0
      0 0 1.0000]

In [ ]:
x_0 = [0,0]
Σ_0 = [1 0 
        0 1]
estimate = Array{Float64}(undef,2,80)


set_state!(kalman, x_0, Σ_0)
logL = 0

for t in 1:size(test_data,2)
    logL = logL + log_likelihood(kalman,test_data[:,t], test_external[:,t])
    update!(kalman, test_data[:,t], test_external[:,t])
    estimate[:,t] = kalman.cur_x_hat
    
    if det(kalman.cur_sigma) == 0
        throw(ErrorException("covariance matrix is non-invertible!"))
        break
    end
end

return kalman, logL